Here I am listing perhaps new resources as discovering new information regarding how to train spaCy has been quite complicated: 

Here we extract common terms within sections (ex. talking about technology) and gets the top x (where x is an an integer)
https://towardsdatascience.com/improving-the-ner-model-with-patent-texts-spacy-prodigy-and-a-bit-of-magic-44c86282ea99

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

https://spacy.io/usage/training

Current config file is set to 

Components: tagger, parser, ner
Hardware: GPU (tranformer)
optimized for: accuracy


////////////////////////////////////////////////////////////////////////////////////////////////////

More food objects that can be for use...
https://fdc.nal.usda.gov/download-datasets.html


In [8]:
# Libraries to use at your disposal, yaspin is just there so you can see that your code IS running,
# 

# !python -m spacy download en_core_web_sm

#!python -m spacy download en_core_web_lg

# !pip3 install yaspin


# When configured the config file, it added 'transformers', need to run this before we can move on...
#!pip3 install spacy[transformers]

     -------------------------------------- 400.7/400.7 MB 9.5 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [10]:
# !python -m spacy init fill-config base_config.cfg config.cfg
'''
The output: 

python -m spacy train config.cfg                   --paths.train ./train.spacy         --paths.dev ./dev.spacy

From old model had this: \
                                 <DIFFERENT------->
python -m spacy train config.cfg --output ./output --paths.train ./train.spacy         --paths.dev ./train.spacy'

For new model perhaps...
                                           <added>                <bit longer                                            <perhaps there for GPU>
python -m spacy train config.cfg --output ./output --paths.train ./training_data.spacy --paths.dev ./training_data.spacy --gpu-id 0

'''

[+] Auto-filled config with all values
[+] Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [40]:
import pandas as pd
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin
import re
import os
from spacy import displacy
from sklearn import metrics
from sklearn.metrics import plot_confusion_matrix
from spacy.util import minibatch, compounding
import random
from pathlib import Path
from yaspin import yaspin
import random # randomizes training data
from spacy.util import filter_spans
import random # randomizes training data
 



FOODKEEPER_PATH = "datasets/FoodKeeper-Data.xls"
TRAINING_DATA_PATH = "datasets/data.csv"
# As I am creating a new model, I will leave the previous one untouched here
MODEL_PATH = "output/model-last"
TEST_DATA_PATH = "datasets/test_data.csv"
REAL_TWITTER_DATA_PATH = "datasets/data.csv"
#STARTING_KEYWORD_COUNT = 10
#TRAINING_LOOP_ITERATIONS = 3
#REQUIRED_KEYWORDS = 3
# MODEL_PATH =  "/dir_model"

pd.options.mode.chained_assignment = None

food_data = pd.read_excel(FOODKEEPER_PATH, sheet_name = "Product")

## printing - bc |||EX: <col1> ID: 1 ~ Category_ID: 7 ~ Name: Butter ~ Name_subtitle: <blank> NaN ~ Keywords: Butter ~ ...
# contains generic info such as the food item, keywords like name: cheese -cheese, parmesan, shredded, grated
# there can be multiple of single name like cheese

# contains patry info like it may be left at room temperature for 1-2 days < cheese>, time it takes for it to expire. refrigeration
# tips and info, max time in there before it goes bad, freezer info like time in there, 
# print("PRINTING food_data excel sheet")
# print(food_data)
# print("DONE ------------------------")

#----------------------------------------------------------------------------------------------------------------------------#
training_data = pd.read_csv(TRAINING_DATA_PATH,index_col = False, header = None)


training_data1 = training_data.sample(frac=0.7)
training_data2 = training_data.drop(training_data1.index)

#ex: A very special Friday evening....many will enjoy a relaxing glass of.....coffee! #lavathejavadog will 
#enjoy the remnants of a crunchie peanut butter jar. Life may throw stuff at us all....letâ€™s all eat peanut butter!
#Enjoy your weekend! https://t.co/JME6ZmhZYN

# print("PRINTING training_data excel sheet")
# print(training_data)
# print("DONE ------------------------")

#----------------------------------------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------------------------------------#
test_data = pd.read_csv(TEST_DATA_PATH)

## printing - bc
# the tweet itself + food column indicating 1: there is a food mention within the tweet or 0: no food in the tweet
# ex:just microwaved a kashi chicken and spinach thing, and put in the Milk dvd. anyone seen it? i bet it's good. i still havent seen slumdog
# food: 1
# print("PRINTING test_data excel sheet")
# print(test_data)
# print("DONE ------------------------")

#----------------------------------------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------------------------------------#
live_tweets = pd.read_csv(REAL_TWITTER_DATA_PATH, header = None)

## printing - bc contains unfiltered tweets

#ex: A very special Friday evening....many will enjoy a relaxing glass of.....coffee! #lavathejavadog will 
#enjoy the remnants of a crunchie peanut butter jar. Life may throw stuff at us all....letâ€™s all eat peanut butter!
#Enjoy your weekend! https://t.co/JME6ZmhZYN

# print("PRINTING live_tweets excel sheet")
# print(live_tweets)
# print("DONE ------------------------")

#----------------------------------------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------------------------------------#


In [14]:
#loop through and count the specific entities
keywords = [] #'chicken', 'milk', 'butter', 'cheese'
sampleData = []

#
#
    
#update rank tweet (based on amount of food entities) to take the counter as a parameter and condense both rankings
def rankTweet(tweet):
    model = spacy.load(MODEL_PATH)
#     tweetKeywords = []
    doc = model(tweet)
    return len(doc.ents)
       
    
# discovers new keywoords
def findNewKeywords(tweet, keywords):
    foodkeeperKeys = foodKeeperInfo() #Gathers all the keywords from the FoodKeeper database
    x = tweet.split() #splits the tweet into a list
    word = "" # current word that may be created
    i = 0
    # looping through the length of the list <word by word>
    while i < len(x):
    #for i in range(len(x)):
        z = 1 # the "jump" to iterate
        if x[i] in foodkeeperKeys:
            word = x[i] # found a word in the database
        try:
            # if the combination of both words x + (x+1) matches a word in the database, make word = that word and increase z
            # to 2 to make the correct iteration
            foundBiWord = x[i] + " " + x[i+1]
            if foundBiWord in foodkeeperKeys: #keywords
                word = foundBiWord
                z = 2
        except:
            pass
        
        try:
            # try a triple combination of words after keyword x was found (x+1) + (x+2). if one found then word = that word and
            # increase the counter by 3 to make correct iteration
            foundTriWord = x[i] + " " + x[i+1] + " " + x[i+2]
            if foundTriWord in foodkeeperKeys: #keywords:
                word = foundTriWord
                z = 3
        except:
            pass
        i += z
        
        # add to the list of keywords (not where the foodKeeperInfo is <contains all keywords from database>)
        # and returns it
        if word not in keywords and word != "":
            keywords.append(word)
    return keywords
    

#Function to find the most common verbs in the tweets
def getCommonVerbs(data):
    import en_core_web_lg
    # individual model here based on the library 'en_core_web_lg'
    nlp2 = spacy.load("en_core_web_lg")
    count = 0
    myVerbs = {}
    # whats data here? 
    for i in range(len(data[0])):
        doc = nlp2(data[0][i])
        for token in doc:
            if token.pos_ == "VERB": # the part of speech, determine if the word itself is a verb like eat
                if token.text in myVerbs: #token.text = word itself to see if it is already in the myVerbs
                    myVerbs[token.text] = myVerbs[token.text] + 1 # increase the counter by 1 <counting number of times found?>
                else:
                    #...Default.stop_words are words like "I", "the", "you" which appear frequently but have no real meaning in the 
                    # spacy model. Thus it is not considered. If it is not that word, then it will be added to the verbs list with
                    # a count of 1 <barely discovered once>
                    if token.text not in nlp2.Defaults.stop_words:
                        myVerbs[token.text] = 1
        
    # sorts the most occuring verbs? 10 of them only?
    topVerbs = dict(sorted(myVerbs.items(), key = lambda item: item[1], reverse=True)[:10])
    return [key for key in topVerbs]
    

 #
#
# Converts a tweet into a training format with each post going through a process of finding keywords and appending entities
#
#
def convertToTrainingFormat(tweet, keywords):
    foodKeeperKeywordsTest = foodKeeperInfo() #Gathers all the keywords from the FoodKeeper database
    x = tweet.split()
    myEnts = {'entities':[]}
    found = False
    i = 0
    foundWords = []
    while i < len(x):
        z = 1
        newWord = ""
        if x[i] in keywords:
            pos = tweet.find(x[i]) # gets the index of where we found the word in the actual tweet
            # (index of where word was found, pos + length of word = end index of the word in tweet, set as a food object <?>)
            # set found to true -> found a word
            y = (pos, pos + len(x[i]), 'FOOD')
            found = True
        if x[i] in foodKeeperKeywordsTest:
            newWord = x[i]
            
        try:
            # try same thing as above but with 2 words
            # ex if we saw something like "sour", in the if before it would not locate it. Here we would locate "sour X" such as
            # "sour cream" without having to trace backwards
            foundBiWord = x[i] + " " + x[i+1]
            if foundBiWord in keywords:
                pos = tweet.find(x[i])
                y = (pos, pos + len(x[i])+len(x[i+1]) + 1, 'FOOD')
                found = True
                z = 2
            if foundBiWord in foodKeeperKeywordsTest:
                newWord = foundBiWord
        except:
            pass
        
        try:
            # try same thing as with 1 word but with 3 words
            foundTriWord = x[i] + " " + x[i+1] + " " + x[i+2]
            if foundTriWord in keywords:
                pos = tweet.find(x[i])
                y = (pos, pos + len(x[i])+len(x[i+1])+len(x[i+2]) + 2, 'FOOD')
                found = True
                z = 3
            if foundTriWord in foodKeeperKeywordsTest:
                newWord = foundTriWord
        except:
            pass
        
        #my own shot for experimental views
        try:
            #4 word attempt
            found_quad_word = x[i] + " " + x[i + 1] + " " + x[i + 2] + " " + x[i + 3]
            if found_quad_word in keywords:
                pos = tweet.find(x[i])
                y = (pos, pos + len(x[i]) + len(x[i + 1]) + len(x[i + 2]) + len(x[i + 3]) + 3, 'FOOD')
                found = True
                z = 4
            if found_quad_word in foodKeeperKeywoordsTest:
                newWord = found_quad_word
        except:
            pass
        # end of trial for my owns sake
        
        # if that word is not currently as an entity, append the word into that list in the dictionary, otherwise continue
        try:
            if y not in myEnts['entities']:
                myEnts['entities'].append(y)    
        except:
            pass

        # declared lines below <if the work is not within the keywordRanker, create a spot for it and set it to counter 1
        #elif increment the counter <meaning we found a word that was already within the keywordRanker>
        if newWord != "" and newWord not in keywordRanker:
            keywordRanker[newWord] = 1
        elif newWord != "" and newWord in keywordRanker:
            keywordRanker[newWord] += 1
        #print(z)
        i += z
        #print(i)
    
    # format the tweet alongside with the entities found within the tweet
    formatted = (tweet, myEnts)
    #print(formatted)
    # means we did find a related word towards the tweet, otherwise return an empty tuple (i think)
    if found:
        return formatted
    else: 
        return ()
     
# food_data contains patry info like it may be left at room temperature for 1-2 days < cheese>, time it takes for it to expire. refrigeration
# tips and info, max time in there before it goes bad, freezer info like time in there,

def foodKeeperInfo():              
    keywords = []
    for word in food_data['Name']: #Butter, Buttermilk, Cheese,...
        word = word.replace(" or ", " ") #word or to " "

        # print("before re.sub[,..]: " + word)
        # word = re.sub('[/]', ' ', word) # sub a character to a ' '
        # print("after: " + word)
        split_word = word.split(",")
        word = re.sub('[/,]', ' ', word) # sub a character to a ' '
        if len(split_word) > 1:
            # still appending as before just in case ex. "shellfish, oysters" -> "shellfish  oysters"
            word = word.lstrip() # rid of left hand side empty characters
            word = word.rstrip() # rid of right hand side empty characters

#             if word.lower() not in keywords: 
#                 keywords.append(word.lower())

            for x in split_word:
                # words like 'butter', 'cheese', would fall into this category
                if len(x.split()) == 1:
                    temp_word = x.lstrip()
                    temp_word = temp_word.rstrip()

                    if temp_word.lower() not in keywords:
                        keywords.append(temp_word.lower()) 
                # words such as 'Cheese,cheddar, swiss,parmesan' split by ',' will be seperated as they would very rarely be found
                # attributed all as a single entity.
                else:
                    temp_word = x.lstrip()
                    temp_word = temp_word.rstrip()

                    var = temp_word.split()
                    # it may be as 'Cheese,cheddar, swiss, and parmesan' which may cause problems as itself is not a FOOD, will be
                    # ignored
                    if var[0].lower() == "and":
                        temp_word = temp_word[3:]
                        temp_word = temp_word.lstrip()
                    
                        
                    if temp_word.lower() not in keywords:
                        keywords.append(temp_word.lower()) 

        else:    
            word = word.lstrip() # rid of left hand side empty characters
            word = word.rstrip() # rid of right hand side empty characters

            # lower cases all the word <if it isn't in keywords> and appends them to the keywords list
            if word.lower() not in keywords: 
                keywords.append(word.lower())

    #print("Total foodkeeper food names: " + str(len(keywords)))        
    #for element in sorted(keywords):
        #print(element)
        
    # return the keywords witih the food_data set into a list formatted accordingly
    return keywords


foodKeeperKeywordsTest = foodKeeperInfo()

# pre-processing of a tweet (can be looked upon deeper such as changing puncuation, though it did not work for me too well).
def preProcess(tweet):
    #Converts a tweet to lowercase, replaces anyusername w/ <USERNAME> and URLS with <URL>
    tweet = tweet.lower()
    tweet = re.sub('@[a-zA-z0-9]*', '', tweet)              # <USERNAME>
    tweet = re.sub('http[a-zA-z0-9./:]*', '', tweet)       # <URL>
    # currently gets rid of ',' and '.' , but not '!', nor '?'.
    # with my changes leave the puncuation of ',' and '.' inside, 
    # results without filtering ',.!?' dropped 88->82
    # now testing filtering ALL punctuation
#     tweet = re.sub('!?,;', '.', tweet)

    # tweet = re.sub('[:;.,-?!]*', '', tweet) #going back to the original with more words (25) + entity counter 2 + get rid of ! and ?

    # original
    tweet = re.sub('[.,-]*', '', tweet)
    
    tweet = re.sub('&amp;', 'and', tweet) # whats &amp;?
    
    return tweet
#
#
# wats this
noEntity= []

keywordRanker = {}

In [15]:
# need to find out what data explicitly means (was not called elsewhere? to what I know - BC)
def information(data):
    myData = {}
    totalEnt=0
    # believe data at data[0] is tweets
    for i in range(len(data[0])):
        #extracts each tweet <processes it as mentioned below> and puts it in the nlp->doc
        # preProcess #Converts a tweet to lowercase, replaces anyusername w/ <USERNAME> and URLS with <URL>
        doc = nlp(preProcess(data[0][i]))
        if len(doc.ents) == 0:
            noEntity.append(preProcess(data[0][i]))
            
        # bottom line was commented out but using it to identity what the length is and what doc.ent contains
        #print(len(doc.ents))
        #print(doc.ents)
        if(len(doc.ents) == 4):
            print(doc)
            
        for entity in doc.ents: 
        #print(entity.label_)
            totalEnt+=1
            if(entity.label_ == 'FOOD'):
                if entity.text in myData:
                    myData[entity.text] += 1
                else:
                    myData[entity.text] = 1
                    
    print("Number of entities found: " + str(len(myData)))
    print(totalEnt)
    for i in sorted(myData, key = myData.get):
        print("Entity: " + i, "Count: " + str(myData[i]), "Density: " + str(format(myData[i]/totalEnt, '.2f')), end = "\n")
    
    
    return myData




for i in range(len(test_data['tweet'])):
    test_data['tweet'][i] = preProcess(test_data['tweet'][i])

y = test_data['food'].tolist()
# print(test_data)
# print(nlp.pipe_names)
    
# renders with displacy an entity (food)
def ent_recognize(text):
    nlp = spacy.load(MODEL_PATH)
    doc = nlp(text)
    displacy.render(doc,style = "ent")
    
# renders with displacy if an entity was discovered (food)
def predict(tweet):
    nlp = spacy.load(MODEL_PATH)
    doc = nlp(str(tweet))
    if doc.ents:
        displacy.render(doc,style = "ent")

# returns a boolean prediction if a food was discovered (Look upon if we should add a more explicit one that counts amount of keywords discovered)
def returnPrediction(tweet):   
    nlp = spacy.load(MODEL_PATH)
    doc = nlp(str(tweet))
    if doc.ents:
        return 1
    else:
        return 0
    
# Calls returnPredictions(tweet) for each individual tweet in the test_data dataset
def get_predictions():
    predictions = []
    for tweet in test_data['tweet'].tolist():
        predictions.append(returnPrediction(tweet))
    return predictions

# evaluations current model by getting current predictions. 
def eval_model(model):
    nlp = spacy.load(model)
    predictions = get_predictions()
    print(metrics.confusion_matrix(y,predictions, labels = [1,0]))
    print(metrics.classification_report(y,predictions, labels = [1,0]))
    
    
def eval_model():
    nlp = spacy.load(MODEL_PATH)
    predictions = get_predictions()
    print(metrics.confusion_matrix(y,predictions, labels = [1,0]))
    print(metrics.classification_report(y,predictions, labels = [1,0]))
    
def show_tp():
    counter = 0
    tweets = test_data['tweet'].tolist()
    predictions = get_predictions()
    for i in range(len(y)):
        if predictions[i] == 1 and y[i] == 1:
            print("True positives:", tweets[i], "\n")
            counter += 1
    print(counter)
    
def show_tn():
    counter = 0
    predictions = get_predictions()
    tweets = test_data['tweet'].tolist()
    for i in range(len(y)):
        if predictions[i] == 0 and y[i] == 0:
            print("True Negative:", tweets[i], "\n")
            counter += 1
    print(counter)
    
def show_fn():
    predictions = get_predictions()
    tweets = test_data['tweet']
    counter = 0
    for i in range(len(y)):
        if predictions[i] == 0 and y[i] == 1:
            print("False Negative:", tweets[i], "\n")
            counter += 1
    print(counter)
    
def show_fp():
    predictions = get_predictions()
    tweets = test_data['tweet'].tolist()
    for i in range(len(y)):
        if predictions[i] == 1 and y[i] == 0:
            print("False Positive:")
            doc = nlp(str(tweets[i]))
            if doc.ents:
                displacy.render(doc,style = "ent")

In [16]:
# save the model to the directory given
def saving(model, directory):
    out_dir = Path(directory)
    if not out_dir.exists():
        print("Directory '" + str(directory) + "' not found! Creating the directory to save model...")
        out_dir.mkdir()
    
    print("writing to ", str(out))
    model.to_disk(out_dir)
    

def load(directory):
    try:
        print("successfully loaded from directory")
        model = spacy.load(directory)
        return model
    except:
        print("No model within the given directory...")
        return


def create(model):
    if (model is not None):
        print("loading model...")
        model = spacy.load(model)
    else:
        print("No model was found... creating a 'en_core_web_sm' model...")
        model = spacy.load("en_core_web_sm")
    
    return model

## Abandoned Model with different approach.
Note that this one was explicitly trying to train directly to spaCy with the given code. However, it was not working as desired. It was left here as scrap code (along with the block of code above with create(), saving(), ect.) if perhaps it will be to any use. For reference, it was following a similar path to this one. https://www.machinelearningplus.com/nlp/training-custom-ner-model-in-spacy/. Note that this was not a direct application towards ours and there were needed changes. Perhaps there are more chagnes there still needed to get this one to work. The current model would be the next function (not training_new(...)). 


In [33]:

# new spacy training model
def training_new(data, model, commonVerbs):
    from spacy.training.example import Example
    
    keywords = [] #foodKeeperInfo()
    oldKeywords = []
    newKeywords = []
    # how many times we plan to iterate
    ITER = 12
    FOOD = "FOOD"
    
    #entityCheckCount controls how many entities are required to 
    #add a Tweet to be trained
    
    # at first it will be 'en_core_web_sm' if it is None it will create the same type anyways...
    nlp = create(model)
    # getting the NER component 
    ner = nlp.get_pipe('ner')
    ner.add_label(FOOD)
    nlp.meta['name'] = 'food_ner'



    #Loop through all the tweets
    #This loop is necessary to get the most common keywords 
    #in the convertToTrainingFormat function




     # to train now that we have appropiate data
    optimizer = nlp.resume_training()
    move_names = list(ner.move_names)

    # to only train the current pipes (can be changed)
    pipes = ["ner", "trf_wordpiecer", "trf_tok2vec"]
    # pipes we do not want to train
    no_pipes = [pipe for pipe in nlp.pipe_names if not pipe not in pipes]
    
    
    #
    #
    #
    
    with nlp.disable_pipes(*no_pipes) :
        
        # Have to see what the arguments within compounding specifically are about...
        sizes = compounding (5.0, 50.0, 1.001) 
        db = DocBin() # create a DocBin object
        myTweets = []
        entityCheckCount = 3
        #Loop through all the tweets
        #This loop is necessary to get the most common keywords 
        #in the convertToTrainingFormat function
        counter = 0
        trainingLoop = True

        while trainingLoop:
            with yaspin() as loading:     

                # getting the training data
                loading.text = "Gathering proper training data on iteration " + str(counter + 1)
                for i in range(len(data[0])): #len(data[0])

                    if counter == 0:
                        x = convertToTrainingFormat(preProcess(data[0][i]), keywords)
                        # x = (tweet, myEnts)
                    #If counter is 1 then there is no model to check so 
                    #a word count is performed
                    elif counter == 1:
                        x = convertToTrainingFormat(preProcess(data[0][i]), keywords)
                        if x!= ():
                            if len(x[1]['entities']) > entityCheckCount:
                                #print("Found tweet", x[0])
                                myTweets.append(x)  
                    else:
                        #Convert each tweet into spacy training format
                        x = convertToTrainingFormat(preProcess(data[0][i]), keywords)
                        checkPassed = False
                        if x != ():
                            #Check the ranking of the tweet

# nlp was model (maybe will have to change to model)
                            # checks the rank with the model it loaded (previous one or the current one that is being trained?)
                            if rankTweet(x[0]) > entityCheckCount:
                                checkTweet = x[0].split()

                                #Check to see if tweet has one of the common verbs
                                for word in checkTweet:
                                    if word in commonVerbs:
                                        checkPassed = True

                                if True: #checkPassed:
                                    #print("Checking rank...")
                                    myTweets.append(x)

                loading.text = "Now training on iteration " + str(counter + 1)               
                random.shuffle(myTweets)
                
                if counter == 0:
                    sortedKeywords =  sorted(keywordRanker, key=keywordRanker.get, reverse=True)
                    
                    # current amount of keywords (food entities) known
                    for i in range(15):
                        keywords.append(sortedKeywords[i])
                
                # actual training occurring here...
                elif counter > 0 and counter < 3:
                    # since there is not as much data in lower iterations of the general training scheme,
                    # make it go through this data more
                    for i in range(20):
                        # shuffle it up
                        random.shuffle(myTweets)
                        losses = {}
                        
#                         for raw_text, entity_offsets in MyTweets:
#                             doc = nlp.make_doc(raw_text)
#                             example = Example.from_
                        
                        # batches the data 
                        # Have to see what the arguments within compounding specifically are about...
                        #sizes = compounding (5.0, 50.0, 1.001) 
                        #batches = minibatch(myTweets, size=2)
                        
                        for batch in minibatch(myTweets, size=2):
                            
                            for text, annotations in batch:
                                #text, annot = zip(*group)

                                doc = nlp.make_doc(text)
                                example = Example.from_dict(doc, annotations)
                                # batch of texts
                                # batch of annotations
                                # update the model with gathered data
                                # dropout - make it harder to memorise data
                                nlp.update([example], sgd=optimizer, drop=0.35)

                
                elif counter > 3:
                    
                    # have it go through the same data (shuffled) with 2 iterations?
                    
                    for i in range(2):
                        # shuffle it up
                        random.shuffle(myTweets)
                        losses = {}
                        # batches the data 
                        #sizes = compounding (5.0, 50.0, 1.001)
                        batches = minibatch(myTweets, size=2)


                        for batch in minibatch(myTweets, size=2):
                            
                            for text, annotations in batch:
                                #text, annot = zip(*group)

                                doc = nlp.make_doc(text)
                                example = Example.from_dict(doc, annotations)
                                # batch of texts
                                # batch of annotations
                                # update the model with gathered data
                                # dropout - make it harder to memorise data
                                nlp.update([example], sgd=optimizer, drop=0.35)

                        
                # saving the model
                loading.text = "Saving the model on iteration " + str(counter + 1)
                saving(nlp, "/dir_model/")
                
                
                print("Total keywords: ", str(len(keywords)))
                print("List of Keywords:\n\n",keywords,"\n\n")
                oldKeywords = len(keywords)
            
                #Look for new keywords
                for element in myTweets:
                    keywords = findNewKeywords(element[0], keywords)
                
            
                #No new keywords are found
                if (oldKeywords == len(keywords)) and counter > 1 and entityCheckCount != 1:
                    entityCheckCount -= 1
                    print("Decreasing entityCheckCount variable by 1")
                    print("entityCheckCount = ", entityCheckCount)
                

                #New keywords are found and entity rank check == 1
                elif (oldKeywords == len(keywords)) and entityCheckCount == 1:
                    trainingLoop = False
            
                eval_model()
                
                print("Total keywords: ", str(len(keywords)))
                print("Total Tweets: ", str(len(myTweets)))
                print("List of Keywords:\n\n",keywords,"\n\n")
                counter += 1
                
    print("Training is completed")
            

## Current Model GPU
This is the current model, it runs on a GPU. If the gpu model is not applicable to you, make sure to use the CPU one instead or find other needs of running it. The main gist of this one is where we write to disk. The main game changer would be to look at the config file for the spaCy model. Elsewhere, much changes cannot be made. Though there is the option of trying a 70/30 split with the current TRAINING dataset to attempt with the 30% dataset testing. 

Note that the main thing to look at would be the config file. More documentation can be looked upon here: https://spacy.io/usage/training#config-custom
If possible, change the config file, look into efficiency for quicker runtime. Currently, the config file is set to accuracy -> long runtime. 

In [48]:
# old model
def trainModel(data, commonVerbs):
    #Initialize all the variables
    keywords = [] #foodKeeperInfo()
    oldKeywords = []
    newKeywords = []
    
    print("Getting the common verbs...")
    print("Common Verbs gathered...", '\n')

    print(commonVerbs)
    
    #entityCheckCount controls how many entities are required to 
    #add a Tweet to be trained
    
    entityCheckCount = 3
    
    counter = 0
    trainingLoop = True
    with yaspin() as loading:
        while trainingLoop:
            loading.text = "Starting training process"
            counterText = "~~~~~~~~~~~~~~~~~"+str(counter)+"~~~~~~~~~~~~~~~~~"
            print(counterText)

            nlp = spacy.blank("en") # load a new spacy model
            try:
                # if we haven't trained anything with a previously trained model. Nothing will load.
                print("getting from path: ", str(MODEL_PATH))
                model = spacy.load(MODEL_PATH)
                print('Model loaded...')
            except:
                print('No model...')

            db = DocBin() # create a DocBin object

            myTweets = []

            #Loop through all the tweets
            #This loop is necessary to get the most common keywords 
            #in the convertToTrainingFormat function


            # getting the training data 
            loading.text = "Gathering training data..."
            for i in range(len(data[0])): #len(data[0])
                #useless if?
    #             if i % 500 == 0:
    #                 print(i)

                if counter == 0:
                    x = convertToTrainingFormat(preProcess(data[0][i]), keywords)
                    # x = (tweet, myEnts)
                #If counter is 1 then there is no model to check so 
                #a word count is performed
                elif counter == 1:
                    x = convertToTrainingFormat(preProcess(data[0][i]), keywords)
                    if x!= ():
                        if len(x[1]['entities']) > entityCheckCount:
                            #print("Found tweet", x[0])
                            myTweets.append(x)  
                else:
                    #Convert each tweet into spacy training format
                    x = convertToTrainingFormat(preProcess(data[0][i]), keywords)
                    checkPassed = False
                    if x != ():
                        #Check the ranking of the tweet

                        # checks the rank with the model it loaded (previous one or the current one that is being trained?)
                        if rankTweet(x[0]) > entityCheckCount:
                            checkTweet = x[0].split()

                            #Check to see if tweet has one of the common verbs
                            for word in checkTweet:
                                if word in commonVerbs:
                                    checkPassed = True

                            if True: #checkPassed:
                                #print("Checking rank...")
                                myTweets.append(x)

                ######
                ###### ^ formatted all the tweets with tweet and entities
                ######



    #         Initialize the keywords

    #         # format the tweet alongside with the entities found within the tweet
    #     formatted = (tweet, myEnts)
    #       Value of the myTweets 
    
            loading.text = "sorting data for docBin file and training if needed on iteration: " + str(counter)
            if counter == 0:  
                # Set keywords to be all keywords found in foodkeeper
    #            keywords = foodKeeperInfo()
                sortedKeywords =  sorted(keywordRanker, key=keywordRanker.get, reverse=True)

                # changing the range of (15) to 30  # now to 22
                for i in range(15): #sortedKeywords
                    keywords.append(sortedKeywords[i])
                #print(sortedKeywords[i], keywordRanker[sortedKeywords[i]])


            elif counter > 0:
                for text, annot in tqdm(myTweets): # data in previous format
                    doc = nlp.make_doc(text) # create doc object from text
                    ents = []



                    # start index of food entity, end index, label = Food, alignment =  (span of all tokens completely within the character span)
                    for start, end, label in annot["entities"]: # add character indexes

                        span = doc.char_span(start, end, label=label, alignment_mode="contract")

                        # none when the character indices dont map to a valid span
                        if span is None:
                            print("Skipping entity")
                        elif ents == []: ents.append(span) 
                        else:
                            #Check to see if any entities are overlapping i.e rice and rice cakes
                            ents.append(span)


    #                         for ent in ents:
    #                             if ent is not None:
    #                                 entLength = ent.end - ent.start
    #                                 if span.start == ent.start or span.end == ent.end:
    #                                     if entLength > (span.end - span.start):
    #                                         continue
    #                                     else:
    #                                         ents.remove(ent)
    #                                         ents.append(span)
    #                                 else:
    #                                     ents.append(span)
                        #if span not in ents: ents.append(span)

                                    #print(span.start, span.end, ents)
                        #put into for loop
    #                 foundEnts = []
    #                 newEnts = []
    #                 for ent in ents:
    #                     if ent.text not in foundEnts:
    #                         foundEnts.append(ent.text)
    #                         newEnts.append(ent)
                    newEnts = filter_spans(ents)              
                    #try:
                    doc.ents = newEnts # label the text with the ents
                        #print(doc)
                    db.add(doc)
                    #except:
                        #print("Error 10: ", doc)



    # Original code that wrote to the disk                      
    #-----------------------------------------------------------------------------------
                # used to be inside ("./training.spacy") (without the _data)
                db.to_disk(".train_data.spacy") # save the docbin object

                #If problems are occuring with the models not appearing
                #ensure that the command is valid, specifically python is the correct
                #PATH variable name on your machine
    # Original code that wrote to the disk           
    # ----------------------------------------------------------------------------------------------------------------------
                #--paths.train should be where the docbin object is saved
                stream = os.popen('python -m spacy train config.cfg --output ./output --paths.train ./training_data.spacy --paths.dev ./training_data.spacy --gpu-id 0')
                print(stream.read())

    # ----------------------------------------------------------------------------------------------------------------------
                loading.text = "Done with current training on iteration: " + str(counter)
                print("Total keywords: ", str(len(keywords)))
                print("List of Keywords:\n\n",keywords,"\n\n")
                oldKeywords = len(keywords)


                #Look for new keywords
                for element in myTweets:
                    keywords = findNewKeywords(element[0], keywords)


                #No new keywords are found
                if (oldKeywords == len(keywords)) and counter > 1 and entityCheckCount != 1:
                    entityCheckCount -= 1
                    print("Decreasing entityCheckCount variable by 1")
                    print("entityCheckCount = ", entityCheckCount)


                #New keywords are found and entity rank check == 1
                elif (oldKeywords == len(keywords)) and entityCheckCount == 1:
                    trainingLoop = False

                eval_model()



            #for element in myTweets:
                #findNewKeywords(element[0], keywords)

            print("Total keywords: ", str(len(keywords)))
            print("Total Tweets: ", str(len(myTweets)))
            print("List of Keywords:\n\n",keywords,"\n\n")
            counter += 1
        
    print('Training Done...')
        
        # spinach < not in dataset> 
        # i like to eat spinach <known to have a keyword
        

In [19]:
foodKeeperInfo() 

['butter',
 'buttermilk',
 'cheese',
 'coffee creamer',
 'cottage cheese',
 'cream cheese',
 'cream',
 'dips',
 'egg substitutes',
 'eggnog',
 'eggs',
 'egg dishes',
 'kefir',
 'margarine',
 'milk',
 'pudding',
 'sour cream',
 'whipped cream',
 'whipped topping',
 'yogurt',
 'beef',
 'lamb',
 'veal',
 'pork',
 'goat',
 'venison',
 'variety meats',
 'bacon',
 'corned beef',
 'ham',
 'hot dogs',
 'sausage',
 'stuffed',
 'raw pork chops',
 'raw kabobs with vegetables',
 'jerky',
 'meat products',
 'retort pouches boxes',
 'chicken',
 'turkey',
 'ground turkey chicken',
 'chicken parts',
 'turkey parts',
 'duckling',
 'goose',
 'pheasant',
 'quail',
 'capon',
 'cornish hens',
 'giblets',
 'raw chicken breasts',
 'turducken',
 'chicken nuggets',
 'patties',
 'cooked poultry dishes',
 'fried chicken',
 'poultry pieces',
 'rotisserie chicken',
 'canned chicken',
 'lean fish',
 'fatty fish',
 'caviar',
 'cooked fish',
 'surimi seafood',
 'scallops',
 'shrimp',
 'crayfish',
 'squid',
 'shucked 

In [25]:
commonVerbs = getCommonVerbs(training_data)

In [49]:
trainModel(data=training_data,commonVerbs=commonVerbs)

Getting the common verbs...
Common Verbs gathered... 

['eat', 'know', 'like', 'think', 'want', 'got', 'love', 'use', 'need', 'add']
⠋ ~~~~~~~~~~~~~~~~~0~~~~~~~~~~~~~~~~~
getting from path:  output/model-last
⠹ Starting training process Model loaded...
⠏ Gathering training data... Total keywords:  15
Total Tweets:  0
List of Keywords:

 ['cheese', 'chicken', 'milk', 'butter', 'juice', 'cream', 'fruit', 'rice', 'water', 'bread', 'garlic', 'sugar', 'chocolate', 'fish', 'salt'] 


~~~~~~~~~~~~~~~~~1~~~~~~~~~~~~~~~~~
⠋ Starting training process  getting from path:  output/model-last
⠹ Starting training process  Model loaded...
⠧ Gathering training data... 

  0%|          | 0/195 [00:00<?, ?it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
⠇ sorting data for docBin file and training if needed on iteration: 1

100%|██████████| 195/195 [00:00<00:00, 2506.40it/s]

⠏ sorting data for docBin file and training if needed on iteration: 1

...

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
⠴ sorting data for docBin file and training if needed on iteration: 5

100%|██████████| 6843/6843 [00:01<00:00, 3736.69it/s]


⠸ sorting data for docBin file and training if needed on iteration: 5 [i] Saving to output directory: output
[i] Using GPU: 0

Total keywords:  419
List of Keywords:

 ['cheese', 'chicken', 'milk', 'butter', 'juice', 'cream', 'fruit', 'rice', 'water', 'bread', 'garlic', 'sugar', 'chocolate', 'fish', 'salt', 'yogurt', 'almond milk', 'greens', 'granola', 'nuts', 'chia seeds', 'whole', 'onions', 'apples', 'peanut butter', 'peaches', 'squash', 'soda', 'flour', 'garlic powder', 'buttermilk', 'potatoes', 'carrots', 'cottage cheese', 'sour cream', 'turkey', 'beans', 'cream cheese', 'bagel', 'pretzels', 'celery', 'white wine', 'fruit cocktail', 'crackers', 'whipped cream', 'margarine', 'ice cream', 'pineapple', 'whipped topping', 'mango', 'sausage', 'pork', 'onion powder', 'dried', 'parsley', 'eggs', 'beef', 'vegetables', 'rotisserie chicken', 'hot dogs', 'dips', 'meats', 'tuna', 'frozen entrees', 'soup', 'bacon', 'pasta', 'pizza', 'fried chicken', 'potato chips', 'cut', 'lean fish', 'cauliflo

⠇ Starting training process                                           Model loaded...


KeyboardInterrupt: 

In [9]:
def getCommonVerbs():
    import en_core_web_lg
    nlp2 = spacy.load("en_core_web_lg")
    count = 0
    myVerbs = {}
    for i in range(10): #(len(training_data)):
        doc = nlp2(training_data[0][i])
        for token in doc:
            print(token, token.pos_)
            if token.pos_ == "VERB":
                if token.text in myVerbs:
                    myVerbs[token.text] = myVerbs[token.text] + 1
                else:
                    if token.text not in nlp2.Defaults.stop_words:
                        myVerbs[token.text] = 1
        
    print(count)
    topVerbs = dict(sorted(myVerbs.items(), key = lambda item: item[1], reverse=True)[:10])
    return [key for key in topVerbs]
# print(getCommonVerbs())

In [ ]:
# print(live_tweets[5][:10])
# print(test_data['tweet'][5])
print("Trying the model indidvidually ")
try:
    nlp = spacy.load(MODEL_PATH)
    print('Model loaded...')
except:
    print('No model...')
    
x = nlp(preProcess("he does like to eat cream cheese"))
if x.doc.ents:
    ent_recognize(preProcess("he does like to eat cream cheese"))
    for i in x.doc.ents:
        print(i)
else:
    print("nothing found...")

In [ ]:
#def findPreviousNextWord(tweets, keywords):
foodKeeperKeywords = foodKeeperInfo()
commonWords = {}
commonPairWords = {}
nlp2 = spacy.load("en_core_web_sm")
for i in range(len(training_data)):
    #Preprocess and remove stop words from each tweet
    myTweet = preProcess(training_data[0][i]).split()
    myTweet = [word for word in myTweet if not word in nlp2.Defaults.stop_words]
    
    for i in range(len(myTweet)):
        if myTweet[i] in foodKeeperKeywords:
            try:
                leftWord = myTweet[i-1]
                if leftWord in commonWords:
                    commonWords[leftWord] += 1
                else:
                    commonWords[leftWord] = 1
            except:
                pass
            try: 
                rightWord = myTweet[i+1]
                if rightWord in commonWords:
                    commonWords[rightWord] += 1
                else:
                    commonWords[rightWord] = 1
            except:
                pass
            try:
                if (leftWord, rightWord) in commonPairWords:
                    commonPairWords[(leftWord,rightWord)] += 1
                else:
                    commonPairWords[(leftWord,rightWord)] = 1
            except:
                pass

#commonWords are words that occur before and after a keyword
commonWords = dict(sorted(commonWords.items(), key = lambda item: item[1], reverse=True))
commonWords = [key for key in commonWords]

#commonPairWords are pairs or words that occur before and after a keyword
commonPairWords = dict(sorted(commonPairWords.items(), key = lambda item: item[1], reverse=True))
# print(training_data[0][1000])
# print(foodKeeperKeywords)

In [ ]:
import re
#count all words and adjacent
def findAdjacentWords(data,keywords):
    allWords = {}
    count = 0
    for tweet in data[0][:10]:
        splitTweet = preProcess(tweet).split()
        for i in range(len(splitTweet)):
            word = splitTweet[i]
            try:
                biword = word + " " + splitTweet[i + 1]
                if biword in keywords:
                    print("Biword:", biword)
                    continue
            except:
                pass
            try:
                triword = word + " " + splitTweet[i + 1] + " " + splitTweet[i + 2]
                if triword in keywords:
                    print("Triword:", triword)
                    continue
            except:
                pass
            
        
    
    '''mySent = "I am eating chicken for breakfast lunch and dinner tonight."
    mySearch = re.search('chicken', mySent)
    mySpan = mySearch.span()
    print(mySent[mySpan[0]:mySpan[1]])
    '''
# findAdjacentWords(training_data, foodKeeperKeywords)